This notebook gives you an end-to-end example on how to get started using Python SDK (preview) to analyze a receipt with Azure Form Recognizer.


## Prerequistes
- Azure subscription - [Create one for free](https://azure.microsoft.com/en-us/free/cognitive-services/)
- [Python 3.x](https://www.python.org/) - Your Python installation should include [pip](https://pip.pypa.io/en/stable/). You can check if you have pip installed by running `pip --version` on the command line. Get pip by installing the latest version of Python.
- Once you have your Azure subscription, [create a Form Recognizer resource](https://ms.portal.azure.com/#create/Microsoft.CognitiveServicesFormRecognizer) in the Azure portal to get your **key** and **endpoint**. After it deploys, click **Go to resource** - You will need the key and endpoint from the resource you create to connect your application to the Form Recognizer API. Later in the quickstart, you will paste your key and endpoint into the code below. You can use the free pricing tier (`F0`) to try the service, and upgrade later to a paid tier (`S0`) for production.

## Setting up - Install the client library
After installing Python, you can install the preview version of Form Recognier client library with:
`pip install azure-ai-formrecognizer --pre`

In [ ]:
#!pip install azure-ai-formrecognizer --pre

## Get the key and endpoint
Refer to the screenshot on how to get the key and endpoint of your Form Recognizer resource.
![How to find endpoint and key](./images/how-to-find-endpoint-and-key.png)

In [1]:
FORMRECOGNIZER_ENDPOINT = "{enter your endpoint}"
FORMRECOGNIZER_KEY = "{enter your key}"


## Authenticate a FormRecognizerClient for document analysis
[FormRecognizerClient](https://docs.microsoft.com/en-us/python/api/azure-ai-formrecognizer/azure.ai.formrecognizer.formrecognizerclient?view=azure-python) is used to query the service to recongize document fields and conent like key-value pairs, tables with prebuilt or custom trained models.

In [3]:
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

# Initiate client with given endpoint and credential
document_analysis_client = DocumentAnalysisClient(endpoint=FORMRECOGNIZER_ENDPOINT, credential=AzureKeyCredential(FORMRECOGNIZER_KEY))

## Start an analyze request for your local files with `begin_analyze_document`

In [8]:
# Read the sample image file into memory and begin analyzing the file
IMAGE_FILE = 'sample-hotel-receipt.png'
with open(IMAGE_FILE, 'rb') as f:
    poller = document_analysis_client.begin_analyze_document(
            "prebuilt-receipt", document=f
        )

# Get the analyze result
receipts = poller.result()
print('Status: {}, Document(s): {}'.format(poller.status(), receipts.documents))

Status: succeeded, Document(s): [AnalyzedDocument(doc_type=prebuilt:receipt, bounding_regions=[BoundingRegion(page_number=1, bounding_box=[Point(x=0.0, y=0.0), Point(x=612.0, y=0.0), Point(x=612.0, y=792.0), Point(x=0.0, y=792.0)])], spans=[DocumentSpan(offset=0, length=701)], fields={'ArrivalDate': DocumentField(value_type=date, value=datetime.date(2021, 3, 27), content=27Mar21, bounding_regions=[BoundingRegion(page_number=1, bounding_box=[Point(x=73.0, y=305.0), Point(x=110.0, y=305.0), Point(x=110.0, y=315.0), Point(x=74.0, y=315.0)])], spans=[DocumentSpan(offset=216, length=7)], confidence=0.986), 'Currency': DocumentField(value_type=string, value='USD', content=None, bounding_regions=[], spans=[], confidence=0.995), 'DepartureDate': DocumentField(value_type=date, value=datetime.date(2021, 3, 28), content=28Mar21, bounding_regions=[BoundingRegion(page_number=1, bounding_box=[Point(x=292.0, y=304.0), Point(x=329.0, y=304.0), Point(x=329.0, y=315.0), Point(x=292.0, y=315.0)])], spans

## You can also analyze files from the web using `begin_analyze_document_from_url`

In [6]:
IMAGE_URL = 'https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/contoso-receipt.png'

# Send request to Form Recognizer service to process data
poller = document_analysis_client.begin_analyze_document_from_url(
        "prebuilt-receipt", document_url=IMAGE_URL
)

# Get the analyze result
receipts = poller.result()
print('Status: {}, Document(s): {}'.format(poller.status(), receipts.documents))

Status: succeeded, Document(s): [AnalyzedDocument(doc_type=prebuilt:receipt, bounding_regions=[BoundingRegion(page_number=1, bounding_box=[Point(x=0.0, y=0.0), Point(x=1688.0, y=0.0), Point(x=1688.0, y=3000.0), Point(x=0.0, y=3000.0)])], spans=[DocumentSpan(offset=0, length=233)], fields={'Items': DocumentField(value_type=list, value=[DocumentField(value_type=dictionary, value={'Name': DocumentField(value_type=string, value='Surface Pro 6', content=Surface Pro 6, bounding_regions=[BoundingRegion(page_number=1, bounding_box=[Point(x=360.0, y=1561.0), Point(x=671.0, y=1563.0), Point(x=671.0, y=1621.0), Point(x=360.0, y=1620.0)])], spans=[DocumentSpan(offset=103, length=13)], confidence=0.975), 'Quantity': DocumentField(value_type=float, value=1.0, content=1, bounding_regions=[BoundingRegion(page_number=1, bounding_box=[Point(x=334.0, y=1560.0), Point(x=349.0, y=1561.0), Point(x=348.0, y=1619.0), Point(x=333.0, y=1619.0)])], spans=[DocumentSpan(offset=101, length=1)], confidence=0.975), '

## Extract information from analyzed result

In [9]:
for idx, receipt in enumerate(receipts.documents):
        print("--------Recognizing receipt #{}--------".format(idx + 1))
        receipt_type = receipt.fields.get("ReceiptType")
        if receipt_type:
            print(
                "Receipt Type: {} has confidence: {}".format(
                    receipt_type.value, receipt_type.confidence
                )
            )
        merchant_name = receipt.fields.get("MerchantName")
        for name, field in receipt.fields.items():
            if name == "Items":
                print("Receipt Items:")
                for id_item, items in enumerate(field.value):
                    print("...Item #{}".format(id_item + 1))
                    for item_name, item in items.value.items():
                        print("......{}: {} has confidence {}".format(item_name, item.value, item.confidence))
            else:
                print("{}: {} has confidence {}".format(name, field.value, field.confidence))

--------Recognizing receipt #1--------
Receipt Type: Hotel has confidence: 0.988
ArrivalDate: 2021-03-27 has confidence 0.986
Currency: USD has confidence 0.995
DepartureDate: 2021-03-28 has confidence 0.986
Receipt Items:
...Item #1
......Category: Room has confidence 0.988
......Date: 2021-03-27 has confidence 0.99
......Description: Room Charge has confidence 0.989
......TotalPrice: 88.0 has confidence 0.99
...Item #2
......Category: Tax has confidence 0.986
......Date: 2021-03-27 has confidence 0.99
......Description: County Tax 6% has confidence 0.987
......TotalPrice: 5.28 has confidence 0.99
...Item #3
......Category: Tax has confidence 0.986
......Date: 2021-03-27 has confidence 0.99
......Description: State Tax 6% has confidence 0.985
......TotalPrice: 5.0 has confidence 0.99
...Item #4
......Category: Other has confidence 0.988
......Date: 2021-03-27 has confidence 0.99
......Description: Daily Parking has confidence 0.989
......TotalPrice: 8.0 has confidence 0.989
...Item #5

## Next steps
- Learn [Receipt concept](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/concept-receipts)
- Explore the [different offerings](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/overview) in Form Recognizer
- Try Form Recognizer with [Form Recognizer Studio](https://docs.microsoft.com/en-us/azure/applied-ai-services/form-recognizer/quickstarts/try-v3-form-recognizer-studio)